In [ ]:
# 기사 목록

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 필요시 활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

# Keywords to search
keywords = [
    "검색어",
]
start_date = "2024-06-01"
end_date = "2024-08-31"
seen_urls = set()

# 웹 드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)

def fetch_search_results(keyword, start_date, end_date):
    page = 1
    results = []
    

    time.sleep(2)  # 페이지 로드 대기

    while True:
        search_url = f"https://www.seoul.co.kr:8888/index.php?scope=&sort=&cpCode=seoul;nownews&period=&sDate={start_date}&eDate={end_date}&keyword={keyword}&iCategory=&pCategory=undefined&pageNum={page}"
        driver.get(search_url)
        print(f"{keyword} {page}")
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 기사 목록 가져오기
        articles = soup.find_all("div", id="list_area")

        if not articles:  # 더 이상 기사가 없을 때 종료
            break

        # 기사 정보 추출
        for section in articles:
            items = section.find_all("dl", class_="article")
            for item in items:
                try:
                    title_tag = item.find("dt").find("a")
                    if title_tag:
                        title = title_tag.text.strip()
                        url = title_tag["href"]
                    else:
                        title = ""
                        url = ""

                    if url in seen_urls:
                        continue
                    seen_urls.add(url)
                    
                    date_element = item.find("span", id="date")
                    date = date_element.text.strip() if date_element else ""
                    date_match = re.search(r"(\d{4})\.\s*(\d{2})\.\s*(\d{2})\s*\((.)\)", date)
                    date = f"{date_match.group(1)}.{date_match.group(2)}.{date_match.group(3)}({date_match.group(4)})" if date_match else ""

                    # 결과 리스트에 추가
                    results.append({
                        'media': '서울신문', 
                        'title': title, 
                        'date': date, 
                        'section': '', 
                        'article_type': '', 
                        'writer': '', 
                        'page': '', 
                        'url': url, 
                        'content': ''  # 컨텐츠 정보 없음
                    })
                except Exception as e:
                    print(f"Error processing article: {e}")
        
        # 다음 페이지로 이동
            page += 1
                          
    return results

def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df.to_excel(filename, index=False)


all_results = []
for keyword in keywords:
    results = fetch_search_results(keyword, start_date, end_date)
    all_results.extend(results)

# Save results to Excel
save_to_excel(all_results, f'1서울신문 {start_date}~{end_date}.xlsx')

# 드라이버 종료
driver.quit()


In [ ]:
# 본문 추출

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

# URL에서 본문과 서브 타이틀, 작가 정보를 스크랩하는 함수 정의
def get_content_and_subtitle_from_url(url):
    try:
        driver.get(url)

        # 여러 가지 클래스명 중 첫 번째로 찾은 본문 추출
        article_classes = [
            '_article_content', 'newsct_article', 'article-txt', 
            'story-news', 'content01', 'viewContentWrap', 
            'atic_txt1', 'articleDiv', 'articleBody'
        ]

        # 각 클래스가 로드될 때까지 시도 (첫 번째로 로드되는 클래스 기다림)
        for class_name in article_classes:
            try:
                WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.CLASS_NAME, class_name))
                )
                break  # 첫 번째로 로드된 클래스가 있으면 루프 종료
            except TimeoutException:
                continue  # 해당 클래스가 로드되지 않았을 경우, 다음 클래스를 시도

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        article_body = None
        for class_name in article_classes:
            article_body = soup.find('div', class_=class_name)
            if article_body:
                break  # 본문을 찾으면 루프 종료

        # 서브 타이틀 추출 (h2.stit 또는 p.stit)
        subtitle_tag = soup.find('h2', class_='stit') or soup.find('p', class_='stit')
        subtitle = ""
        if subtitle_tag:
            subtitle = subtitle_tag.get_text(separator='\n', strip=True)
            subtitle_tag.decompose()  # 부제를 본문에서 제거

        # 본문에서 불필요한 태그 제거
        writer = ""
        if article_body:
            # 작가 정보 추출 (div.byline)
            byline_tag = article_body.find('div', class_='byline')
            if byline_tag:
                writer = byline_tag.get_text(strip=True)
                byline_tag.decompose()  # 본문에서 작가 정보를 제거
        
            for tag in article_body.find_all(class_=['v_photoarea', 'photoWrap', 'photo_caption', 'v_photo_caption', 'modal']) + article_body.find_all(id='img'):
                tag.decompose()

            content = article_body.get_text(separator='\n', strip=True)
        else:
            content = ""

        return content, subtitle, writer

    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return None, None, None

# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트와 로그 파일
error_log = []
total_articles = len(df)

# 각 행의 URL에서 본문 내용을 스크랩하여 content, sub-title, writer 컬럼에 저장합니다.
for index, row in df.iterrows():
    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content, subtitle, writer = get_content_and_subtitle_from_url(row['url'])
    
    if content is None:
        error_log.append(row['url'])
    else:
        if content.startswith('='):
            content = "'" + content        
        df.at[index, 'content'] = content  
        df.at[index, 'sub-title'] = subtitle  # 서브 타이틀 컬럼 추가
        df.at[index, 'writer'] = writer  # 기존 writer 컬럼에 저장

    # 100개마다 파일 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

# 변경된 데이터를 파일로 저장합니다.
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)

print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")

# 에러 로그 파일 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")
    print(f"에러 로그가 저장되었습니다: {error_log_path}")

      
